In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_binary = pd.read_csv(r'.\TrafficLabelling\data_binary.csv')
data_multiclass = pd.read_csv(r'.\TrafficLabelling\data_multi_class.csv')

In [3]:
data_binary.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80,192.168.10.5,55054,6,7/7/2017 3:30,109,1,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80,192.168.10.5,55055,6,7/7/2017 3:30,52,1,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443,192.168.10.16,46236,6,7/7/2017 3:30,34,1,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443,192.168.10.5,54863,6,7/7/2017 3:30,3,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [4]:
pd.set_option('display.max_rows', None)  # hiển thị tất cả dòng
pd.set_option('display.max_columns', None)  # hiển thị tất cả cột
pd.set_option("display.float_format", "{:.2f}".format)

In [5]:
summary = data_binary.describe().T[['min','max','mean']]
# Hiển thị toàn bộ bảng (không bị cắt)
pd.set_option('display.max_rows', None)  # hiển thị tất cả dòng
pd.set_option('display.max_columns', None)  # hiển thị tất cả cột

# Hiển thị số ở dạng thường, làm tròn 2 chữ số thập phân
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

print(summary)


                                           min              max          mean
Source Port                               0.00        65,535.00     41,128.86
Destination Port                          0.00        65,535.00      8,071.48
Protocol                                  0.00            17.00          9.88
Flow Duration                           -13.00   119,999,998.00 14,785,663.93
Total Fwd Packets                         1.00       219,759.00          9.36
Total Backward Packets                    0.00       291,922.00         10.39
Total Length of Fwd Packets               0.00    12,900,000.00        549.30
Total Length of Bwd Packets               0.00   655,453,030.00     16,162.64
Fwd Packet Length Max                     0.00        24,820.00        207.60
Fwd Packet Length Min                     0.00         2,325.00         18.71
Fwd Packet Length Mean                    0.00         5,940.86         58.20
Fwd Packet Length Std                     0.00         7,125.60 

In [6]:
print("Tổng số giá trị thiếu:", data_binary.isnull().sum().sum())
print("Tổng số giá trị thiếu:", data_multiclass.isnull().sum().sum())

Tổng số giá trị thiếu: 0
Tổng số giá trị thiếu: 0


In [7]:
all_zero_cols = [col for col in data_binary.columns if (data_binary[col].nunique() == 1 and data_binary[col].iloc[0] == 0)]

print("Các cột toàn 0:")
for col in all_zero_cols:
    print(f"- {col}")


Các cột toàn 0:
- Bwd PSH Flags
- Bwd URG Flags
- Fwd Avg Bytes/Bulk
- Fwd Avg Packets/Bulk
- Fwd Avg Bulk Rate
- Bwd Avg Bytes/Bulk
- Bwd Avg Packets/Bulk
- Bwd Avg Bulk Rate


In [8]:
data_binary = data_binary.drop(columns=all_zero_cols)
data_multiclass = data_multiclass.drop(columns=all_zero_cols)

In [9]:
numeric_cols = data_binary.select_dtypes(include="number")
neg_count = (numeric_cols < 0).sum()
neg_percent = neg_count / len(data_binary) * 100
result = pd.DataFrame({
    "Số âm": neg_count,
    "Tỷ lệ %": neg_percent
})
result["Tỷ lệ %"] = result["Tỷ lệ %"].map(lambda x: f"{x:.10f}")
print(result)


                               Số âm        Tỷ lệ %
Source Port                        0   0.0000000000
Destination Port                   0   0.0000000000
Protocol                           0   0.0000000000
Flow Duration                    115   0.0040625376
Total Fwd Packets                  0   0.0000000000
Total Backward Packets             0   0.0000000000
Total Length of Fwd Packets        0   0.0000000000
Total Length of Bwd Packets        0   0.0000000000
Fwd Packet Length Max              0   0.0000000000
Fwd Packet Length Min              0   0.0000000000
Fwd Packet Length Mean             0   0.0000000000
Fwd Packet Length Std              0   0.0000000000
Bwd Packet Length Max              0   0.0000000000
Bwd Packet Length Min              0   0.0000000000
Bwd Packet Length Mean             0   0.0000000000
Bwd Packet Length Std              0   0.0000000000
Flow Bytes/s                      85   0.0030027452
Flow Packets/s                   115   0.0040625376
Flow IAT Mea

In [ ]:
data_binary = data_binary.drop(columns=["Init_Win_bytes_backward", "Init_Win_bytes_forward","Timestamp"])
data_multiclass = data_multiclass.drop(columns=["Init_Win_bytes_backward", "Init_Win_bytes_forward","Timestamp"])

In [11]:
# Chọn các cột số
numeric_cols = data_binary.select_dtypes(include="number")

# Giữ lại chỉ những hàng mà tất cả giá trị số đều >= 0
data_binary = data_binary[(numeric_cols >= 0).all(axis=1)]
data_multiclass = data_multiclass[(numeric_cols >= 0).all(axis=1)]


In [12]:
print(data_binary["Label"].value_counts())
print(data_multiclass["Label"].value_counts())

Label
BENIGN    2270365
ATTACK     557452
Name: count, dtype: int64
Label
BENIGN            2270365
DOS_DDOS           380505
PROBE_PORTSCAN     158930
CRED_WEB            16009
R2L_U2R_OTHER        2008
Name: count, dtype: int64


In [13]:
data_binary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2827817 entries, 0 to 2830742
Data columns (total 75 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Flow ID                      object 
 1   Source IP                    object 
 2   Source Port                  int64  
 3   Destination IP               object 
 4   Destination Port             int64  
 5   Protocol                     int64  
 6   Timestamp                    object 
 7   Flow Duration                int64  
 8   Total Fwd Packets            int64  
 9   Total Backward Packets       int64  
 10  Total Length of Fwd Packets  float64
 11  Total Length of Bwd Packets  float64
 12  Fwd Packet Length Max        float64
 13  Fwd Packet Length Min        float64
 14  Fwd Packet Length Mean       float64
 15  Fwd Packet Length Std        float64
 16  Bwd Packet Length Max        float64
 17  Bwd Packet Length Min        float64
 18  Bwd Packet Length Mean       float64
 19  Bwd P

In [14]:
data_binary.describe().T

,count,mean,std,min,25%,50%,75%,max
Source Port,"2,827,817.00","41,125.28","22,299.29",0.00,"32,748.00","50,944.00","58,418.00","65,535.00"
Destination Port,"2,827,817.00","8,076.71","18,288.82",0.00,53.00,80.00,443.00,"65,535.00"
Protocol,"2,827,817.00",9.88,5.26,0.00,6.00,6.00,17.00,17.00
Flow Duration,"2,827,817.00","14,742,371.30","33,606,562.84",0.00,155.00,"31,299.00","3,141,118.00","119,999,998.00"
Total Fwd Packets,"2,827,817.00",5.73,74.33,1.00,2.00,2.00,5.00,"112,141.00"
Total Backward Packets,"2,827,817.00",5.55,102.92,0.00,1.00,2.00,4.00,"157,388.00"
Total Length of Fwd Packets,"2,827,817.00",527.25,"9,116.29",0.00,12.00,62.00,187.00,"12,900,000.00"
Total Length of Bwd Packets,"2,827,817.00","5,342.22","250,413.32",0.00,0.00,122.00,479.00,"399,000,000.00"
Fwd Packet Length Max,"2,827,817.00",207.05,716.34,0.00,6.00,37.00,80.00,"24,820.00"
Fwd Packet Length Min,"2,827,817.00",18.73,60.37,0.00,0.00,2.00,36.00,"2,325.00"


In [15]:
numeric_cols = data_binary.select_dtypes(include="number")

corr_matrix = numeric_cols.corr().abs()

# Lấy phần tam giác trên (để tránh trùng lặp và loại đường chéo = 1)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Tìm các cặp có corr > 0.95
high_corr = [
    (col, row, upper.loc[row, col])
    for col in upper.columns
    for row in upper.index
    if pd.notna(upper.loc[row, col]) and upper.loc[row, col] > 0.90
]

# Chuyển thành DataFrame để xem dễ hơn
high_corr_df = pd.DataFrame(high_corr, columns=["Feature 1", "Feature 2", "Correlation"])

print(high_corr_df.sort_values(by="Correlation", ascending=False))

                      Feature 1                    Feature 2  Correlation
35               CWE Flag Count                Fwd URG Flags         1.00
52          Fwd Header Length.1            Fwd Header Length         1.00
54          Subflow Fwd Packets            Total Fwd Packets         1.00
41         Avg Fwd Segment Size       Fwd Packet Length Mean         1.00
62          Subflow Bwd Packets       Total Backward Packets         1.00
34               SYN Flag Count                Fwd PSH Flags         1.00
43         Avg Bwd Segment Size       Bwd Packet Length Mean         1.00
70            Subflow Bwd Bytes  Total Length of Bwd Packets         1.00
60            Subflow Fwd Bytes  Total Length of Fwd Packets         1.00
8                 Fwd IAT Total                Flow Duration         1.00
12                  Fwd IAT Max                 Flow IAT Max         1.00
39          Average Packet Size           Packet Length Mean         1.00
36               ECE Flag Count       

In [16]:
drop_collums=["CWE Flag Count","SYN Flag Count","Subflow Bwd Packets","Avg Fwd Segment Size","Fwd Header Length.1",
              "Subflow Fwd Packets","Avg Bwd Segment Size","Subflow Bwd Bytes","Subflow Fwd Bytes","Fwd IAT Total",
              "Fwd IAT Max","Average Packet Size","ECE Flag Count","Bwd Header Length","Idle Min","Idle Mean","Total Backward Packets",
              "Fwd Packets/s","Fwd Header Length","Fwd Packet Length Std","act_data_pkt_fwd","Total Length of Bwd Packets",
              "Idle Max","Packet Length Std","Bwd Packet Length Std","Bwd Packet Length Mean","Packet Length Variance","Active Min",
              "Bwd Packet Length Max","Flow IAT Max","Bwd IAT Min","Max Packet Length","Fwd IAT Mean"]
data_binary = data_binary.drop(columns=drop_collums)
data_multiclass = data_multiclass.drop(columns=drop_collums)

In [17]:
# data_binary.to_csv(r'.\TrafficLabelling\data_binary.csv', index=False)
# data_multiclass.to_csv(r'.\TrafficLabelling\data_multi_class.csv', index=False)

In [18]:
data_binary.describe().T

,count,mean,std,min,25%,50%,75%,max
Source Port,"2,827,817.00","41,125.28","22,299.29",0.00,"32,748.00","50,944.00","58,418.00","65,535.00"
Destination Port,"2,827,817.00","8,076.71","18,288.82",0.00,53.00,80.00,443.00,"65,535.00"
Protocol,"2,827,817.00",9.88,5.26,0.00,6.00,6.00,17.00,17.00
Flow Duration,"2,827,817.00","14,742,371.30","33,606,562.84",0.00,155.00,"31,299.00","3,141,118.00","119,999,998.00"
Total Fwd Packets,"2,827,817.00",5.73,74.33,1.00,2.00,2.00,5.00,"112,141.00"
Total Length of Fwd Packets,"2,827,817.00",527.25,"9,116.29",0.00,12.00,62.00,187.00,"12,900,000.00"
Fwd Packet Length Max,"2,827,817.00",207.05,716.34,0.00,6.00,37.00,80.00,"24,820.00"
Fwd Packet Length Min,"2,827,817.00",18.73,60.37,0.00,0.00,2.00,36.00,"2,325.00"
Fwd Packet Length Mean,"2,827,817.00",58.18,185.99,0.00,6.00,34.00,50.00,"5,940.86"
Bwd Packet Length Min,"2,827,817.00",41.09,68.88,0.00,0.00,0.00,77.00,"2,896.00"


### Scale

In [21]:
data_binary.shape

(2827817, 42)

In [26]:
# 1. Nhóm feature dạng count/flag (0/1 hoặc nhỏ)
flag_features = ['Fwd PSH Flags', 'Fwd URG Flags', 'FIN Flag Count',
                 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count']

# 2. Nhóm feature có outlier lớn
outlier_features = ['Flow Duration', 'Flow Bytes/s', 'Flow Packets/s',
                    'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Min',
                    'Fwd IAT Std', 'Fwd IAT Min',
                    'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                    'Active Mean', 'Active Std', 'Active Max', 'Idle Std']

# 3. Nhóm còn lại (Port, Length, Ratio, Packet stats…)
other_features = [col for col in data_binary.columns if col not in flag_features + outlier_features +["Label"] ]
